### Python-struct
1. 解析和打包二进制数据
2. **[The notebook consult from here](https://docs.python.org/3.6/library/struct.html)**

---

1. 模块的作用
    
    * 模块的函数的作用在于转换 `Python` 和 `C` 之间的数据
    * `Python` 的 `bytes` 类型的字符串数据被用来接收和保存 `C` 结构体的数据并使用格式控制符号 `format` 的字符串描述数据在 `C` 结构体中的布局模式
    * 可以用来控制二进制数据的存储或者网络信息的存储
    * `format` 格式控制字符串表示了对 `C` 语言结构体的格式描述和扩展的 `Python` 的值的转换
    
2. 主要函数
    1. `struct.pack(fmt, v1, v2, ...)` : 返回的是一个字节字符串，是参数按照fmt数据格式组合而成。
    2. `struct.unpack(fmt, buffer)` : 解析字节字符串到 `Python` 的标准类型
    3. `struct.clacsize(fmt)` : 按照字节字符串的解析格式计算 `Python` 的数据大小
    
3. 格式控制符号 `format`
    1. 作用 : 在解析和构建数据的时候提供一种合适布局格式
    2. 字节序，大小和对齐 **是 `format` 的第一个字符串**
    
    ---

| Character | Byte order | Size | Alignment |
        | --------- | ---------- | ---- | --------- |
        | @ 	    | native 	 | native | native |
        | = 	    | native 	 | standard | none |
        | < 	    | little-endian | standard | none |
        | > 	    | big-endian | standard | none |
        | ! 	    | network (= big-endian) | standard | none |
      
      
 * 如果第一个字符不是上述的几个的话，默认是 `@`
 * `native` 是大端还是小段模式取决于当前的系统，使用 `sys.byteorder` 查看当前的系统是大端还是小端模式
 * `standard` 模式主要取决于之后的格式控制字符
 * `network` 模式表示网络信息传输中，一般都使用大端传输模式
 
    ---
    
    格式控制字符串表示了对 `C` 语言的数据类型到 `Python` 数据类型的转换方式
    
| Format |	C Type |	Python type |	Standard size |	Notes |
| --- | --- | --- | --- | --- | 
| x |	pad byte |	no value | | | 	  	 
| c |	char |	bytes of length | 1 |	1 	 |
| b |	signed char |	integer |	1 |	(1),(3) |
| B |	unsigned char |	integer |	1 |	(3) |
| ? |  	_Bool |	bool |	1 |	(1) |
| h |	short |	integer |	2 |	(3) |
| H |	unsigned short |	integer |	2 |	(3) |
| i |	int |	integer |	4 |	(3) |
| I |	unsigned int |	integer |	4 |	(3) |
| l |	long 	|integer |	4 |	(3)|
| L |	unsigned long |	integer |	4 |	(3)|
|q| 	long long |	integer |	8 	|(2), (3)|
|Q| 	unsigned long long| 	integer |	8 |	(2), (3)|
|n| 	ssize_t |	integer |	  |	(4)|
|N| 	size_t 	|integer| | 	  	(4)|
|e| 	(7) |	float |	2 |	(5)|
|f| 	float| 	float| 	4| 	(5)|
|d| 	double| 	float| 	8| 	(5)|
|s| 	char[]| 	bytes 	  	 |||
|p| 	char[]| 	bytes 	  	 |||
|P| 	void * |	integer |	  |	(6)|

* 使用了 `standard` 模式的话，使用的是上式的字节大小标准，但是如果是 `native` 的话是和当前的系统平台有关的
* 上述的格式控制字符前可能会出现一个整形数字比如 `4h`,代表重复 `hhhh`， 这一点对于 `s` / `p` 也是适用的，但是表示的是数组的长度，默认是1

In [3]:
# 导入包
import struct

In [16]:
# 下面的所有例子都是 native 模式下
big = struct.pack('>hhl', 1, 2, 3)
little = struct.pack('<hhl', 1, 2, 3)
# 可以看出来大端模式中，高位字节在低地址中，低位字节在低地址中
print(big)
print(little)

# 要正确的使用解析模式才会解析出正确的数字，否则只会解析出错误的数组
# 解析正确
unbig = struct.unpack('>hhl', big)
print(unbig)
# 解析错误, 小端解释的话,刚好将 \x00\x01 -> 100000000 -> 256
unbig = struct.unpack('<hhl', big)
print(unbig)

# 计算大小, @方式使用了内存对齐的方式，一个 4 字节
print(struct.calcsize('hhl'), struct.calcsize('<hhl'), struct.calcsize('>hhl'))

b'\x00\x01\x00\x02\x00\x00\x00\x03'
b'\x01\x00\x02\x00\x03\x00\x00\x00'
(1, 2, 3)
(256, 512, 50331648)
16 8 8


In [19]:
# namedtuple 
record = b'raymond   \x32\x12\x08\x01\x08'
name, serialnum, school, gradelevel = struct.unpack('<10sHHb', record)
from collections import namedtuple
Student = namedtuple('Student', 'name serialnum school gradelevel')

# 大端模式
# \x12\x32 -> 0001001000110010 -> 4658
# \x01\x08 -> 0000000100001000 -> 264
# \x08 -> 字符是 8
Student._make(struct.unpack('<10sHHb', record))

Student(name=b'raymond   ', serialnum=4658, school=264, gradelevel=8)

In [20]:
# 内存对齐
print(struct.pack('ci', b'*', 0x12131415))
print(struct.pack('ic', 0x12131415, b'*'))
print(struct.calcsize('ci'))
print(struct.calcsize('ic'))

b'*\x00\x00\x00\x15\x14\x13\x12'
b'\x15\x14\x13\x12*'
8
5
